I have placed some of the less interesting code in the `convert_book_to_text.py` file to try to keep the notebook flowing. Note that almost all of the code in that file was generated by ChatGPT

In [8]:
##for the reload of all the functions in the file convert_pdf_to_text.py
from importlib import reload
import convert_book_to_text

reload(convert_book_to_text)
from convert_book_to_text import find_chapter_pages, \
                                 find_page_starting_with,\
                                 extract_text_from_pages_inclusive, \
                                 build_chapter_dictionary, \
                                 remove_chapter_header, \
                                 chunk_text, \
                                 find_bad_csv_files


For copyright reasons I have not included the ebook in the repo. You should buy your own copy if you want to run this code

In [9]:
from pathlib import Path
import PyPDF2

input_file = Path('./inputs/Truth to Power_ My Three Years Inside Eskom.pdf')
input_file = Path(input_file)
# Instantiate PdfReader object
pdf = PyPDF2.PdfReader(input_file)


In [10]:
# This is very specific to the format of the book I am processing. Not making this general is a design decision. This repo needs back and forward with a human
# to work so spending too much time generalising functions seems like a waste of time.

# Note: This is quite slow. It takes about 30 seconds to run on my machine.
pages_with_numbers = find_chapter_pages(pdf)
pages_with_numbers.append(find_page_starting_with(pdf, "Index"))
# Create a dictionary where each chapter is saved with its start and end page
chapter_start_and_end = build_chapter_dictionary(pages_with_numbers)

The following cell is for illustration. It allows the user to see what is happening in the main loop below, but for a single chapter

In [19]:
chapter_number = 2
start = chapter_start_and_end[chapter_number]["start_page"]
end = chapter_start_and_end[chapter_number]["end_page"]
print("processing chapter: ", str(chapter_number) + " from page " + str(start) + " to page " + str(end))
text = remove_chapter_header(extract_text_from_pages_inclusive(pdf, start, end))
chunked_text = chunk_text(text, 800)

processing chapter:  2 from page 12 to page 18


In [33]:
from importlib import reload
import sentiment_analysis

reload(sentiment_analysis)
from sentiment_analysis import save_people_named_in_chapters, \
                               combine_csvs, \
                               aggregate_sentiment


The main loop. Chapters start at 1 and end at 31. chapter_range_end should be set to the number after the last chapter you want to process so, if you are processing the last chapter, chapter 31, only you would set `chapter_range_start = 31` and `chapter_range_end = 32`.

The output of this is a list of CSV files in the folder `./output/`. A separate file is created for each chunk. 

The function sets a chunk size of 800 words. There is no science in determining how long this should be but we need to ensure there are sufficient tokens for the input, the system command and the output. I started with a chunk size of 1000 words but I found there were times where GPT3.5 would truncate the output. 800 seemed reasonable. 

In [21]:
chapter_range_start = 1
chapter_range_end = 3
output_folder = "./sentiment_detail"
save_people_named_in_chapters(chapter_range_start=chapter_range_start, \
                              chapter_range_end=chapter_range_end, \
                              chapter_start_and_end = chapter_start_and_end,\
                              pdf = pdf, \
                              output_folder = output_folder)

processing chapter:  1 from page 9 to page 11
The file sentiment_detail\chapter_1_0.csv already exists so skipping it
The file sentiment_detail\chapter_1_1.csv already exists so skipping it
processing chapter:  2 from page 12 to page 18
The file sentiment_detail\chapter_2_0.csv already exists so skipping it
The file sentiment_detail\chapter_2_1.csv already exists so skipping it
The file sentiment_detail\chapter_2_2.csv already exists so skipping it
The file sentiment_detail\chapter_2_3.csv already exists so skipping it
The file sentiment_detail\chapter_2_4.csv already exists so skipping it


It is hard to ensure the LLM creates output in a consistent format (it doesn't matter how nicely you ask). There are two approaches to work with this, one would be to run a second ChatGPT call to check if the output was formatted correctly and if not to fix it, the other is just to do it manually. Given that the number of times GPT provided output in a different format, I opted just to fix the issues manually.

In [22]:
find_bad_csv_files(output_folder)

[('all.csv', 'Columns mismatch'), ('all_v1.csv', 'Columns mismatch')]

Once all the output files are formatted correctly, they are concatenated into a single list

In [30]:
import pandas as pd
# output the number of rows in df
df = combine_csvs(output_folder)
print(df.shape)
df.to_csv(output_folder + "/combined/all.csv", encoding='utf-8', index=False)

(1063, 5)


The real manual work begins here. The consolidated list needs to be cleaned. Even though the system prompt asked only for information on people, the AI added organisations. Sometimes it would only list a person by their first or last name, sometimes both. Sometimes the list would include a title, other times it did not. I opted for a manual exercise to remove items I was not interested in and to ensure the names that were left were realitvely standard. 

In [31]:
import pandas as pd
manual_list = pd.read_csv(output_folder + "/combined/all_v1.csv", encoding='utf-8')
# count the number of unique names in the column manual_list["Name"]
manual_list["Name"].nunique()

280

In [34]:
name = "Gwede Mantashe"
output_folder = "./aggregate"
output = aggregate_sentiment(name = name, output_folder = output_folder, manual_list = manual_list)
output

Found 22 entries for Gwede Mantashe
